<a href="https://colab.research.google.com/github/sdhar2020/Visualization-Exploratory-Analysis/blob/master/Mobile_App_to_deal_with_Lottery_Addiction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Settings
import pandas as pd

# Important functions

In [1]:
def factorial(n):
    final_product = 1
    for i in range(n, 0, -1):
        final_product *= i
    return final_product

def permutation(n, k):
    numerator = factorial(n)
    denominator = factorial(n-k)
    return numerator/denominator

def combinations(n,k):
    combination = permutation(n,k)/ factorial(k)
    return(combination)

# Introduction

A medical institute that aims to prevent and treat gambling addictions wants to build a dedicated mobile app to help lottery addicts better estimate their chances of winning. The institute has a team of engineers that will build the app, but they need us to create the logical core of the app and calculate probabilities.

For the first version of the app, they want us to focus on the 6/49 lottery and build functions that enable users about lottery outcomes



## Loading and Exploring Data

In [5]:
df1 = pd.read_csv('https://raw.githubusercontent.com/sdhar2020/Visualization-Exploratory-Analysis/master/datasets_32665_42513_649.csv')

In [6]:
df1.describe()

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
count,3665.0,3665.000000,3665.000000,3665.000000,3665.000000,3665.000000,3665.000000,3665.000000,3665.000000,3665.000000
mean,649.0,1819.494952,0.030832,7.327694,14.568076,21.890859,28.978445,36.162619,43.099045,24.599454
std,0.0,1039.239544,0.237984,5.811669,7.556939,8.170073,8.069724,7.190960,5.506424,14.360038
min,649.0,1.000000,0.000000,1.000000,2.000000,3.000000,4.000000,11.000000,13.000000,0.000000
25%,649.0,917.000000,0.000000,3.000000,9.000000,16.000000,23.000000,31.000000,40.000000,12.000000
50%,649.0,1833.000000,0.000000,6.000000,14.000000,22.000000,30.000000,37.000000,45.000000,25.000000
75%,649.0,2749.000000,0.000000,10.000000,20.000000,28.000000,35.000000,42.000000,47.000000,37.000000
max,649.0,3591.000000,3.000000,38.000000,43.000000,45.000000,47.000000,48.000000,49.000000,49.000000


In [7]:
df1.isnull().sum()

PRODUCT            0
DRAW NUMBER        0
SEQUENCE NUMBER    0
DRAW DATE          0
NUMBER DRAWN 1     0
NUMBER DRAWN 2     0
NUMBER DRAWN 3     0
NUMBER DRAWN 4     0
NUMBER DRAWN 5     0
NUMBER DRAWN 6     0
BONUS NUMBER       0
dtype: int64

# Big Prize

we want players to be able to calculate the probability of winning the big prize with the various numbers they play on a single ticket (for each ticket a player chooses six numbers out of 49). So, we'll start by building a function that calculates the probability of winning the big prize for any given ticket

In [8]:
def one_ticket_probability(user_numbers):
    
    n_combinations = combinations(49, 6)
    probability_one_ticket = 1/n_combinations
    percentage_form = probability_one_ticket * 100
    
    print('''Your chances to win the big prize with the numbers {} are {:.7f}%.
In other words, you have a 1 in {:,} chances to win.'''.format(user_numbers,
                    percentage_form, int(n_combinations)))

In [9]:
test_input_1 = [2, 43, 22, 23, 11, 5]
one_ticket_probability(test_input_1)

Your chances to win the big prize with the numbers [2, 43, 22, 23, 11, 5] are 0.0000072%.
In other words, you have a 1 in 13,983,816 chances to win.


In [10]:
test_input_2 = [9, 26, 41, 7, 15, 6]
one_ticket_probability(test_input_2)

Your chances to win the big prize with the numbers [9, 26, 41, 7, 15, 6] are 0.0000072%.
In other words, you have a 1 in 13,983,816 chances to win.


In [11]:
df1.columns

Index(['PRODUCT', 'DRAW NUMBER', 'SEQUENCE NUMBER', 'DRAW DATE',
       'NUMBER DRAWN 1', 'NUMBER DRAWN 2', 'NUMBER DRAWN 3', 'NUMBER DRAWN 4',
       'NUMBER DRAWN 5', 'NUMBER DRAWN 6', 'BONUS NUMBER'],
      dtype='object')

In [12]:
def extract_numbers(x):
  tkt_num_cols =['NUMBER DRAWN 1', 'NUMBER DRAWN 2', 'NUMBER DRAWN 3', 'NUMBER DRAWN 4','NUMBER DRAWN 5', 'NUMBER DRAWN 6']
  x = x[tkt_num_cols]
  x = set(x.values)
  return(x)


In [13]:
win_lott_nums= df1.apply(extract_numbers, axis = 1)

In [14]:
def check_historical_occurrence(user_numbers, historical_numbers):   
    '''
    user_numbers: a Python list
    historical numbers: a pandas Series
    '''
    user_numbers_set = set(user_numbers)
    check_occurrence = historical_numbers == user_numbers_set
    n_occurrences = check_occurrence.sum()
    
    if n_occurrences == 0:
        print('''The combination {} has never occured.
This doesn't mean it's more likely to occur now. Your chances to win the big prize in the next drawing using the combination {} are 0.0000072%.
In other words, you have a 1 in 13,983,816 chances to win.'''.format(user_numbers, user_numbers))
        
    else:
        print('''The number of times combination {} has occured in the past is {}.
Your chances to win the big prize in the next drawing using the combination {} are 0.0000072%.
In other words, you have a 1 in 13,983,816 chances to win.'''.format(user_numbers, n_occurrences,
                                                                            user_numbers))

Testing for possible input strings

In [15]:
test_input_3 = [33, 36, 37, 39, 8, 41]

In [16]:
check_historical_occurrence(test_input_3, win_lott_nums)

The number of times combination [33, 36, 37, 39, 8, 41] has occured in the past is 1.
Your chances to win the big prize in the next drawing using the combination [33, 36, 37, 39, 8, 41] are 0.0000072%.
In other words, you have a 1 in 13,983,816 chances to win.


In [17]:

test_input_4 = [3, 2, 44, 22, 1, 44]
check_historical_occurrence(test_input_4, win_lott_nums)

The combination [3, 2, 44, 22, 1, 44] has never occured.
This doesn't mean it's more likely to occur now. Your chances to win the big prize in the next drawing using the combination [3, 2, 44, 22, 1, 44] are 0.0000072%.
In other words, you have a 1 in 13,983,816 chances to win.


# Multi-ticket Probability
For the first version of the app, users should also be able to find the probability of winning if they play multiple different tickets. For instance, someone might intend to play 15 different tickets and they want to know the probability of winning the big prize.

The engineering team wants us to be aware of the following details when we're writing the function:

The user will input the number of different tickets they want to play (without inputting the specific combinations they intend to play).
Our function will see an integer between 1 and 13,983,816 (the maximum number of different tickets).
The function should print information about the probability of winning the big prize depending on the number of different tickets played.
The multi_ticket_probability() function below takes in the number of tickets and prints probability information depending on the input.

In [18]:
def multi_ticket_probability(n_tickets):
    
    n_combinations = combinations(49, 6)
    
    probability = n_tickets / n_combinations
    percentage_form = probability * 100
    
    if n_tickets == 1:
        print('''Your chances to win the big prize with one ticket are {:.6f}%.
In other words, you have a 1 in {:,} chances to win.'''.format(percentage_form, int(n_combinations)))
    
    else:
        combinations_simplified = round(n_combinations / n_tickets)   
        print('''Your chances to win the big prize with {:,} different tickets are {:.6f}%.
In other words, you have a 1 in {:,} chances to win.'''.format(n_tickets, percentage_form,
                                                               combinations_simplified))

Below, we run a couple of tests for our function.



In [19]:
test_inputs = [1, 10, 100, 10000, 1000000, 6991908, 13983816]

for test_input in test_inputs:
    multi_ticket_probability(test_input)
    print('------------------------') # output delimiter

Your chances to win the big prize with one ticket are 0.000007%.
In other words, you have a 1 in 13,983,816 chances to win.
------------------------
Your chances to win the big prize with 10 different tickets are 0.000072%.
In other words, you have a 1 in 1,398,382 chances to win.
------------------------
Your chances to win the big prize with 100 different tickets are 0.000715%.
In other words, you have a 1 in 139,838 chances to win.
------------------------
Your chances to win the big prize with 10,000 different tickets are 0.071511%.
In other words, you have a 1 in 1,398 chances to win.
------------------------
Your chances to win the big prize with 1,000,000 different tickets are 7.151124%.
In other words, you have a 1 in 14 chances to win.
------------------------
Your chances to win the big prize with 6,991,908 different tickets are 50.000000%.
In other words, you have a 1 in 2 chances to win.
------------------------
Your chances to win the big prize with 13,983,816 different ti

Less Winning Numbers — Function
In most 6/49 lotteries, there are smaller prizes if a player's ticket match two, three, four, or five of the six numbers drawn. This means that players might be interested in finding out the probability of having two, three, four, or five winning numbers — for the first version of the app, users should be able to find those probabilities.

These are the details we need to be aware of when we write a function to make the calculations of those probabilities possible:

- Inside the app, the user inputs:
   - six different numbers from 1 to 49; and
   - an integer between 2 and 5 that represents the number of winning numbers expected

- Our function prints information about the probability of having a certain number of winning numbers

To calculate the probabilities, we tell the engineering team that the specific combination on the ticket is irrelevant and we only need the integer between 2 and 5 representing the number of winning numbers expected. Consequently, we will write a function named probability_less_6() which takes in an integer and prints information about the chances of winning depending on the value of that integer.

In [20]:
def probability_less_6(n_winning_numbers):
    
    n_combinations_ticket = combinations(6, n_winning_numbers)
    n_combinations_remaining = combinations(49 - n_winning_numbers,
                                           6 - n_winning_numbers)
    successful_outcomes = n_combinations_ticket * n_combinations_remaining
    n_combinations_total = combinations(49, 6)
    
    probability = successful_outcomes / n_combinations_total
    probability_percentage = probability * 100
    
    combinations_simplified = round(n_combinations_total/successful_outcomes)
    
    print('''Your chances of having {} winning numbers with this ticket are {:.6f}%.
In other words, you have a 1 in {:,} chances to win.'''.format(n_winning_numbers, probability_percentage,
                                                               int(combinations_simplified)))

In [21]:
for test_input in [2, 3, 4, 5]:
    probability_less_6(test_input)
    print('--------------------------') # output delimiter

Your chances of having 2 winning numbers with this ticket are 19.132653%.
In other words, you have a 1 in 5 chances to win.
--------------------------
Your chances of having 3 winning numbers with this ticket are 2.171081%.
In other words, you have a 1 in 46 chances to win.
--------------------------
Your chances of having 4 winning numbers with this ticket are 0.106194%.
In other words, you have a 1 in 942 chances to win.
--------------------------
Your chances of having 5 winning numbers with this ticket are 0.001888%.
In other words, you have a 1 in 52,969 chances to win.
--------------------------
